In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pydot
from IPython.display import display, SVG, clear_output

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.all import (Parser, StartMeshcat, DiagramBuilder,
                        MeshcatVisualizerCpp, JacobianWrtVariable,
                        MakeRenderEngineVtk, RenderEngineVtkParams,
                        Simulator, CoulombFriction, HalfSpace, 
                        RotationMatrix, RotationalInertia)
from pydrake.geometry import (
    Box,
    Cylinder
)

from pydrake.multibody.tree import (
    PrismaticJoint,
    UnitInertia,
    SpatialInertia,
    RevoluteJoint,
    FixedOffsetFrame,
    WeldJoint
)

from manipulation.meshcat_cpp_utils import MeshcatJointSliders
from manipulation.scenarios import AddMultibodyTriad
import modern_robotics as mr

In [3]:
meshcat = StartMeshcat()

Meshcat is now available at http://localhost:7001


In [ ]:
# Build robot from code
builder = DiagramBuilder()
step_size = 1e-4
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, step_size)

# Some parameters
L0 = 0.5
L1 = 0.7
L2 = 0.4

h = 0.06
w = 0.08
r = 0.05

m = 1

RGBA_Color = [0.5, 0.5, 0.5, 1]
mu = 0.4

my_model_instance = plant.AddModelInstance("my_robot")

inertia_link_0 = SpatialInertia.MakeFromCentralInertia(
    m, [0, 0, L0/2], RotationalInertia(m*(3*r**2+L0**2)/12, m*(3*r**2+L0**2)/12, m*r**2/2))
inertia_link_1 = SpatialInertia.MakeFromCentralInertia(
    m, [0, 0, L1/2], RotationalInertia(m*(w**2+L1**2)/12, m*(h**2+L1**2)/12, m*(h**2+w**2)/12))
inertia_link_2 = SpatialInertia.MakeFromCentralInertia(
    m, [0, 0, L2/2], RotationalInertia(m*(w**2+L2**2)/12, m*(h**2+L2**2)/12, m*(h**2+w**2)/12))

link_0 = plant.AddRigidBody(
    "link_0", my_model_instance, inertia_link_0)
link_1 = plant.AddRigidBody(
    "link_1", my_model_instance, inertia_link_1)
link_2 = plant.AddRigidBody(
    "link_2", my_model_instance, inertia_link_2)


plant.RegisterVisualGeometry(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0/2]),
    Cylinder(r, L0),
    "link_0",
    RGBA_Color)
plant.RegisterVisualGeometry(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1/2]),
    Box(h, w, L1),
    "link_1",
    RGBA_Color)
plant.RegisterVisualGeometry(
    link_2,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L2/2]),
    Box(h, w, L2),
    "link_2",
    RGBA_Color)

frame_on_link_0 = plant.AddFrame(FixedOffsetFrame(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0])))

frame_on_link_1 = plant.AddFrame(FixedOffsetFrame(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1])))

plant.AddJoint(RevoluteJoint(
    name="joint_0_to_1", frame_on_parent=frame_on_link_0,
    frame_on_child=link_1.body_frame(), axis=[1, 0, 0]))

plant.AddJoint(RevoluteJoint(
    name="joint_1_to_2", frame_on_parent=frame_on_link_1,
    frame_on_child=link_2.body_frame(), axis=[1, 0, 0]))

plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=link_0.body_frame(),
    X_PC=RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]))

# Draw RGB frames for visualization
for body_name in ["link_0", "link_1", "link_2"]:
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph, 0.20, 0.008)

# Finalize and visualize
plant.Finalize()

renderer_name = "renderer"
scene_graph.AddRenderer(
    renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))

meshcat.Delete()
meshcat_vis = MeshcatVisualizerCpp.AddToBuilder(
    builder, scene_graph, meshcat)

diagram = builder.Build()

diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
plant.SetPositions(plant_context, plant.GetModelInstanceByName("my_robot"),
                   [0.1, 0.2])  # theta1, theta2

diagram.Publish(diagram_context)

simulator = Simulator(diagram, diagram_context)
simulator.set_publish_every_time_step(True)
simulator.set_target_realtime_rate(1)
simulator.Initialize()
diagram.Publish(diagram_context)
simulator.AdvanceTo(5)


In [ ]:
# please write your own simulator below
